In [5]:
def run_pandas():
    """Implementação completa com Pandas"""
    import time
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.impute import KNNImputer
    
    start_time = time.time()


    #---------------------------------------------------------------------------------------------
    # Parte 1: Modelo de ML (ML_Pisa) completa os valores omissos do dataset pisa_2006-2018
    #---------------------------------------------------------------------------------------------

    pisa = pd.read_csv("pisa_2006-2018.csv")
    pisa.replace('NA', np.nan, inplace=True)
    pisa['Score'] = pd.to_numeric(pisa['Score'], errors='coerce')

    # Codificação
    le = LabelEncoder()
    pisa['Country_encoded'] = le.fit_transform(pisa['Country'])
    pisa['Subject_encoded'] = le.fit_transform(pisa['Subject'])

    # Modelo
    train = pisa.dropna(subset=['Score'])
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(train[['Year', 'Subject_encoded', 'Country_encoded']], train['Score'])

    # Previsão
    missing = pisa[pisa['Score'].isna()]
    if not missing.empty:
        pred = model.predict(missing[['Year', 'Subject_encoded', 'Country_encoded']])
        pisa.loc[pisa['Score'].isna(), 'Score'] = np.round(pred)

    pisa.to_csv("pisa_imputed_pandas.csv", index=False)


    #---------------------------------------------------------------------------------------------
    # Parte 2: Junção dos datasets
    #---------------------------------------------------------------------------------------------

    # Carregar os datasets
    pisa_imputed = pd.read_csv("pisa_imputed_pandas.csv")
    gdp = pd.read_csv("GDP.csv")

    # Corrigir possíveis espaços nos nomes das colunas
    gdp.columns = gdp.columns.str.strip()

    # Filtrar apenas os anos de interesse no dataset PISA
    pisa_imputed = pisa_imputed[pisa_imputed['Year'].isin([2012, 2015, 2018])]

    # Filtrar colunas relevantes do GDP
    gdp = gdp[['Country', 'Country Code', '2012', '2015', '2018']]

    # Obter lista de países comuns
    paises_comuns = set(pisa_imputed['Country']).intersection(set(gdp['Country']))

    # Filtrar os datasets para manter apenas países comuns
    gdp_filtrado = gdp[gdp['Country'].isin(paises_comuns)].copy()
    pisa_filtrado = pisa_imputed[pisa_imputed['Country'].isin(paises_comuns)].copy()

    # Transformar GDP de wide para long
    gdp_long = gdp_filtrado.melt(
        id_vars=['Country', 'Country Code'],
        value_vars=['2012', '2015', '2018'],
        var_name='Year',
        value_name='GDP'
    )

    # Garantir que o ano está como inteiro
    gdp_long['Year'] = gdp_long['Year'].astype(int)

    # Transformar PISA de long para wide (Subject → colunas)
    pisa_wide = pisa_filtrado[['Country', 'Year', 'Subject', 'Score']].pivot_table(
        index=['Country', 'Year'],
        columns='Subject',
        values='Score'
    ).reset_index()

    # Juntar os datasets
    dados_combinados = pd.merge(gdp_long, pisa_wide, on=['Country', 'Year'], how='left')

    # Reestruturar para wide por ano
    pisa_e_GDP = dados_combinados.pivot(
        index=['Country', 'Country Code'],
        columns='Year',
        values=['GDP', 'Maths', 'Science', 'Reading']
    )

    # Ajustar nomes das colunas para formato mais legível
    pisa_e_GDP.columns = [f"{var}_{year}" for var, year in pisa_e_GDP.columns]
    pisa_e_GDP = pisa_e_GDP.reset_index()
    pisa_e_GDP = pisa_e_GDP.rename(columns={
        'Country Code': 'Country_code'
    })

    pisa_e_GDP.to_csv("pisa_e_GDP.csv", index=False)


    #---------------------------------------------------------------------------------------------
    # Parte 3: Processamento do GDP_PISA (ML_GDP_Pisa)
    #---------------------------------------------------------------------------------------------

    dt = pd.read_csv("pisa_e_GDP.csv")

    # Tratamento de NAs
    print("\n[Pandas] Valores NaN antes:", dt.isna().sum().sum())

    # Salvar máscara de valores ausentes para usar depois
    missing_mask = dt.isna()

    colunas_numericas = dt.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    dt_scaled = dt.copy()
    dt_scaled[colunas_numericas] = scaler.fit_transform(dt[colunas_numericas])

    imputer = KNNImputer(n_neighbors=5)
    dt_imputed_scaled = dt_scaled.copy()
    dt_imputed_scaled[colunas_numericas] = imputer.fit_transform(dt_scaled[colunas_numericas])

    dt_imputed = dt.copy()
    dt_imputed[colunas_numericas] = scaler.inverse_transform(dt_imputed_scaled[colunas_numericas])

    # Formatação
    colunas_gdp = ["GDP_2012", "GDP_2015", "GDP_2018"]
    dt_imputed[colunas_gdp] = dt_imputed[colunas_gdp].round(5)

    # Correção: Aplicar arredondamento e conversão para int apenas nas células que eram NaN
    # Exclui colunas GDP que já foram tratadas separadamente
    colunas_para_arredondar = [col for col in colunas_numericas if col not in colunas_gdp]

    for col in colunas_para_arredondar:
        if missing_mask[col].any():
            dt_imputed.loc[missing_mask[col], col] = dt_imputed.loc[missing_mask[col], col].round(0).astype(int)

    print("[Pandas] Valores NaN após:", dt_imputed.isna().sum().sum())


    #---------------------------------------------------------------------------------------------
    # Parte 4: Adicionar as médias dos anos e das disciplinas
    #---------------------------------------------------------------------------------------------

    # Criar novas colunas com médias das disciplinas arredondadas a 3 casas decimais
    dt_imputed['Means_Maths'] = dt_imputed[['Maths_2012', 'Maths_2015', 'Maths_2018']].mean(axis=1, skipna=True).round(3)
    dt_imputed['Means_Science'] = dt_imputed[['Science_2012', 'Science_2015', 'Science_2018']].mean(axis=1, skipna=True).round(3)
    dt_imputed['Means_Reading'] = dt_imputed[['Reading_2012', 'Reading_2015', 'Reading_2018']].mean(axis=1, skipna=True).round(3)

    dt_imputed['Mean_2012'] = dt_imputed[[c for c in dt_imputed.columns if '2012' in c and 'GDP' not in c]].mean(axis=1).round(3)
    dt_imputed['Mean_2015'] = dt_imputed[[c for c in dt_imputed.columns if '2015' in c and 'GDP' not in c]].mean(axis=1).round(3)
    dt_imputed['Mean_2018'] = dt_imputed[[c for c in dt_imputed.columns if '2018' in c and 'GDP' not in c]].mean(axis=1).round(3)


    #---------------------------------------------------------------------------------------------
    # Parte 5: Adicionar variações relativas (%) entre anos
    #---------------------------------------------------------------------------------------------

    # Variação percentual do GDP
    dt_imputed['GDP_change_2012_2015'] = ((dt_imputed['GDP_2015'] - dt_imputed['GDP_2012']) / dt_imputed['GDP_2012'] * 100).round(2)
    dt_imputed['GDP_change_2015_2018'] = ((dt_imputed['GDP_2018'] - dt_imputed['GDP_2015']) / dt_imputed['GDP_2015'] * 100).round(2)

    # Variação percentual das médias PISA
    dt_imputed['PISA_change_2012_2015'] = ((dt_imputed['Mean_2015'] - dt_imputed['Mean_2012']) / dt_imputed['Mean_2012'] * 100).round(2)
    dt_imputed['PISA_change_2015_2018'] = ((dt_imputed['Mean_2018'] - dt_imputed['Mean_2015']) / dt_imputed['Mean_2015'] * 100).round(2)


    #---------------------------------------------------------------------------------------------
    # Parte 6: Rácio GDP / PISA Score por ano
    #---------------------------------------------------------------------------------------------

    dt_imputed['GDP_per_PISA_2012'] = (dt_imputed['GDP_2012'] / dt_imputed['Mean_2012']).round(4)
    dt_imputed['GDP_per_PISA_2015'] = (dt_imputed['GDP_2015'] / dt_imputed['Mean_2015']).round(4)
    dt_imputed['GDP_per_PISA_2018'] = (dt_imputed['GDP_2018'] / dt_imputed['Mean_2018']).round(4)


    #---------------------------------------------------------------------------------------------
    # Parte 7: Diferença absoluta das pontuações PISA e do GDP entre anos
    #---------------------------------------------------------------------------------------------

    # Diferença absoluta
    dt_imputed['GDP_diff_2012_2015'] = (dt_imputed['GDP_2015'] - dt_imputed['GDP_2012']).round(3)
    dt_imputed['GDP_diff_2015_2018'] = (dt_imputed['GDP_2018'] - dt_imputed['GDP_2015']).round(3)

    dt_imputed['PISA_diff_2012_2015'] = (dt_imputed['Mean_2015'] - dt_imputed['Mean_2012']).round(3)
    dt_imputed['PISA_diff_2015_2018'] = (dt_imputed['Mean_2018'] - dt_imputed['Mean_2015']).round(3)

    dt_imputed.to_excel("Dataset_final_pandas.xlsx", index=False)


    return f"Tempo de execução Pandas: {time.time() - start_time} segundos"


In [3]:
#---------------------------------------------------------------------------------------------
# Usando o PySpark
#---------------------------------------------------------------------------------------------

def run_pyspark():
    """Implementação completa com PySpark"""
    import time
    import numpy as np
    from pyspark.sql import SparkSession
    from pyspark.sql import functions as F
    from pyspark.sql.types import DoubleType, IntegerType
    from pyspark.ml.feature import StringIndexer
    from pyspark.ml.regression import RandomForestRegressor
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml import Pipeline
    from pyspark.ml.evaluation import RegressionEvaluator
    from pyspark.ml.feature import Imputer
    from pyspark.ml.feature import StandardScaler
    from pyspark.sql.window import Window
    
    start_time = time.time()
    
    # Iniciar sessão Spark
    spark = SparkSession.builder \
        .appName("PISA_GDP_Analysis") \
        .config("spark.driver.memory", "4g") \
        .getOrCreate()
    
    #---------------------------------------------------------------------------------------------
    # Parte 1: Modelo de ML (ML_Pisa)
    #---------------------------------------------------------------------------------------------
    
    # Carregar dados
    pisa = spark.read.csv("pisa_2006-2018.csv", header=True, inferSchema=True)
    
    # Converter Score para double e substituir 'NA' por null
    pisa = pisa.withColumn("Score", 
                          F.when(F.col("Score") == "NA", None)
                          .otherwise(F.col("Score").cast(DoubleType())))
    
    # Codificação com StringIndexer
    countryIndexer = StringIndexer(inputCol="Country", outputCol="Country_encoded")
    subjectIndexer = StringIndexer(inputCol="Subject", outputCol="Subject_encoded")
    
    # Criar pipeline para codificação
    indexPipeline = Pipeline(stages=[countryIndexer, subjectIndexer])
    indexModel = indexPipeline.fit(pisa)
    pisaIndexed = indexModel.transform(pisa)
    
    # Dividir dados para treino (sem NaN) e previsão (com NaN)
    train = pisaIndexed.filter(pisaIndexed.Score.isNotNull())
    missing = pisaIndexed.filter(pisaIndexed.Score.isNull())
    
    # Preparar features para o modelo
    featureAssembler = VectorAssembler(
        inputCols=["Year", "Subject_encoded", "Country_encoded"],
        outputCol="features"
    )
    
    # Criar e treinar o modelo
    rf = RandomForestRegressor(
        labelCol="Score",
        featuresCol="features",
        numTrees=100,
        seed=42,
        maxDepth=10
    )
    
    pipeline = Pipeline(stages=[featureAssembler, rf])
    model = pipeline.fit(train)
    
    # Fazer previsões para valores em falta
    if missing.count() > 0:
        predictions = model.transform(missing)
        predictions = predictions.withColumn("Score", F.round(F.col("prediction")))
        
        # Unir os dados treinados com as previsões
        pisaImputed = train.select("Country", "Subject", "Year", "Score") \
            .union(predictions.select("Country", "Subject", "Year", "Score"))
    else:
        pisaImputed = train.select("Country", "Subject", "Year", "Score")
    
    # Salvar o dataset imputado
    pisaImputed.write.mode("overwrite").csv("pisa_imputed_pyspark", header=True)
    
    # Para compatibilidade com os próximos passos, vamos criar um arquivo CSV 
    # (normalmente não seria necessário em um pipeline completo Spark)
    pisaImputed.toPandas().to_csv("pisa_imputed_pyspark.csv", index=False)
    
    #---------------------------------------------------------------------------------------------
    # Parte 2: Junção dos datasets
    #---------------------------------------------------------------------------------------------
    
    # Carregar os datasets
    pisa_imputed = spark.read.csv("pisa_imputed_pyspark.csv", header=True, inferSchema=True)
    gdp = spark.read.csv("GDP.csv", header=True, inferSchema=True)
    
    # Remover espaços nos nomes das colunas (PySpark não tem um método direto como o pandas)
    for col_name in gdp.columns:
        if ' ' in col_name:
            gdp = gdp.withColumnRenamed(col_name, col_name.strip())
    
    # Filtrar apenas os anos de interesse no dataset PISA
    pisa_imputed = pisa_imputed.filter(F.col("Year").isin([2012, 2015, 2018]))
    
    # Selecionar colunas relevantes do GDP
    gdp = gdp.select("Country", "Country Code", "2012", "2015", "2018")
    
    # Obter lista de países comuns
    paises_pisa = set([row["Country"] for row in pisa_imputed.select("Country").distinct().collect()])
    paises_gdp = set([row["Country"] for row in gdp.select("Country").distinct().collect()])
    paises_comuns = paises_pisa.intersection(paises_gdp)
    
    # Filtrar os datasets para manter apenas países comuns
    gdp_filtrado = gdp.filter(F.col("Country").isin(list(paises_comuns)))
    pisa_filtrado = pisa_imputed.filter(F.col("Country").isin(list(paises_comuns)))
    
    # Transformar GDP de wide para long
    gdp_long = gdp_filtrado.select(
        "Country", 
        F.col("Country Code").alias("Country_Code"),
        F.lit("2012").alias("Year"), 
        F.col("2012").alias("GDP")
    ).union(
        gdp_filtrado.select(
            "Country", 
            F.col("Country Code").alias("Country_Code"),
            F.lit("2015").alias("Year"), 
            F.col("2015").alias("GDP")
        )
    ).union(
        gdp_filtrado.select(
            "Country", 
            F.col("Country Code").alias("Country_Code"),
            F.lit("2018").alias("Year"), 
            F.col("2018").alias("GDP")
        )
    )
    
    # Garantir que o ano está como inteiro
    gdp_long = gdp_long.withColumn("Year", F.col("Year").cast(IntegerType()))
    
    # Transformar PISA de long para wide (Subject → colunas)
    # Primeiro agrupar por País e Ano
    pisa_pivot = pisa_filtrado.groupBy("Country", "Year").pivot("Subject").agg(F.first("Score"))
    
    # Juntar os datasets
    dados_combinados = gdp_long.join(pisa_pivot, on=["Country", "Year"], how="left")
    
    # Reestruturar para wide por ano (usando expressões mais complexas no PySpark)
    # Primeiro criar uma coluna para cada combinação "variável_ano"
    dados_wide = dados_combinados
    
    # Criar colunas GDP por ano
    for year in [2012, 2015, 2018]:
        dados_wide = dados_wide.withColumn(
            f"GDP_{year}", 
            F.when(F.col("Year") == year, F.col("GDP")).otherwise(None)
        )
        
        # Criar colunas para cada matéria e ano
        for subject in ["Maths", "Science", "Reading"]:
            dados_wide = dados_wide.withColumn(
                f"{subject}_{year}", 
                F.when(F.col("Year") == year, F.col(subject)).otherwise(None)
            )
    
    # Agrupar por país para colocar os valores nos mesmos registros
    window_spec = Window.partitionBy("Country", "Country_Code")
    
    pisa_e_GDP = dados_wide.groupBy("Country", "Country_Code").agg(
        F.max("GDP_2012").alias("GDP_2012"),
        F.max("GDP_2015").alias("GDP_2015"),
        F.max("GDP_2018").alias("GDP_2018"),
        F.max("Maths_2012").alias("Maths_2012"),
        F.max("Maths_2015").alias("Maths_2015"),
        F.max("Maths_2018").alias("Maths_2018"),
        F.max("Science_2012").alias("Science_2012"),
        F.max("Science_2015").alias("Science_2015"),
        F.max("Science_2018").alias("Science_2018"),
        F.max("Reading_2012").alias("Reading_2012"),
        F.max("Reading_2015").alias("Reading_2015"),
        F.max("Reading_2018").alias("Reading_2018")
    )
    
    # Renomear colunas
    pisa_e_GDP = pisa_e_GDP.withColumnRenamed("Country", "Nacao").withColumnRenamed("Country_Code", "Country_code")
    
    # Salvar como CSV
    pisa_e_GDP.toPandas().to_csv("pisa_e_GDP_pyspark.csv", index=False)
    
    #---------------------------------------------------------------------------------------------
    # Parte 3: Processamento do GDP_PISA (ML_GDP_Pisa)
    #---------------------------------------------------------------------------------------------
    
    # Carregar dados
    dt = spark.read.csv("pisa_e_GDP_pyspark.csv", header=True, inferSchema=True)
    
    # Contar valores NaN antes da imputação
    nan_count_before = sum([dt.filter(F.col(c).isNull()).count() for c in dt.columns])
    print(f"\n[PySpark] Valores NaN antes: {nan_count_before}")
    
    # Identificar colunas numéricas
    colunas_numericas = [field.name for field in dt.schema.fields 
                          if isinstance(field.dataType, (DoubleType, IntegerType))]
    
    # Criar uma cópia do DataFrame com as colunas originais para depois recuperar
    dt_original = dt
    
    # Padronização
    assembler = VectorAssembler(inputCols=colunas_numericas, outputCol="features", handleInvalid="keep")
    scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
    
    pipeline = Pipeline(stages=[assembler, scaler])
    scalerModel = pipeline.fit(dt)
    dt_scaled = scalerModel.transform(dt)
    
    # Imputação de valores ausentes (PySpark não tem KNNImputer, usando Imputer simples)
    imputer = Imputer(
        inputCols=colunas_numericas,
        outputCols=[f"{c}_imputed" for c in colunas_numericas],
        strategy="mean"  # Usando média como estratégia
    )
    
    imputer_model = imputer.fit(dt)
    dt_imputed = imputer_model.transform(dt)
    
    # Substituir colunas originais pelas imputadas
    for col in colunas_numericas:
        dt_imputed = dt_imputed.withColumn(col, F.col(f"{col}_imputed"))
        dt_imputed = dt_imputed.drop(f"{col}_imputed")
    
    # Formatar colunas GDP com 5 casas decimais
    colunas_gdp = ["GDP_2012", "GDP_2015", "GDP_2018"]
    for col in colunas_gdp:
        dt_imputed = dt_imputed.withColumn(col, F.round(F.col(col), 5))
    
    # Arredondar e converter para inteiro outras colunas numéricas
    colunas_para_arredondar = [col for col in colunas_numericas if col not in colunas_gdp]
    for col in colunas_para_arredondar:
        dt_imputed = dt_imputed.withColumn(
            col, 
            F.when(
                F.col(col).isNull().cast("boolean"), 
                F.round(F.col(col), 0).cast(IntegerType())
            ).otherwise(F.col(col))
        )
    
    # Contar valores NaN após a imputação
    nan_count_after = sum([dt_imputed.filter(F.col(c).isNull()).count() for c in dt_imputed.columns])
    print(f"[PySpark] Valores NaN após: {nan_count_after}")
    
    #---------------------------------------------------------------------------------------------
    # Parte 4: Adicionar as médias dos anos e das disciplinas
    #---------------------------------------------------------------------------------------------
    
    # Criar novas colunas com médias das disciplinas arredondadas a 3 casas decimais
    dt_imputed = dt_imputed.withColumn(
        "Means_Maths",
        F.round(F.array_avg(F.array("Maths_2012", "Maths_2015", "Maths_2018")), 3)
    )
    
    dt_imputed = dt_imputed.withColumn(
        "Means_Science",
        F.round(F.array_avg(F.array("Science_2012", "Science_2015", "Science_2018")), 3)
    )
    
    dt_imputed = dt_imputed.withColumn(
        "Means_Reading",
        F.round(F.array_avg(F.array("Reading_2012", "Reading_2015", "Reading_2018")), 3)
    )
    
    # Médias por ano (excluindo GDP)
    dt_imputed = dt_imputed.withColumn(
        "Mean_2012",
        F.round(F.array_avg(F.array("Maths_2012", "Science_2012", "Reading_2012")), 3)
    )
    
    dt_imputed = dt_imputed.withColumn(
        "Mean_2015",
        F.round(F.array_avg(F.array("Maths_2015", "Science_2015", "Reading_2015")), 3)
    )
    
    dt_imputed = dt_imputed.withColumn(
        "Mean_2018",
        F.round(F.array_avg(F.array("Maths_2018", "Science_2018", "Reading_2018")), 3)
    )
    
    # Salvar resultado final
    dt_imputed.toPandas().to_excel("Dataset_final_pyspark.xlsx", index=False)
    
    # Encerrar a sessão Spark
    spark.stop()
    
    return f"Tempo de execução PySpark: {time.time() - start_time} segundos"

In [10]:
run_pandas()

run_pandas()

[Pandas] Valores NaN antes: 70
[Pandas] Valores NaN após: 0
Tempo de execução Pandas: 0.6846075057983398 segundos

[PySpark] Valores NaN antes: 70
[PySpark] Valores NaN após: 0
Tempo de execução PySpark: 5.5756016855123312 segundos
